목차

1. 각 변수에 대한 탐색
    - 자세한 설명
    - 목적 변수와의 관계 도출

2. 상관관계 탐색
    - 스피어만 상관 계수
    
3. 데이터 전처리
    1. 이상치 제거
    2. 변수 정규화
    3. 변수 수정

4. 변수 생성




참조 : https://rstudio-pubs-static.s3.amazonaws.com/155304_cc51f448116744069664b35e7762999f.html3

https://geodacenter.github.io/data-and-lab//KingCounty-HouseSales2015/


* ID : 집을 구분하는 번호
* date : 집을 구매한 날짜
* price : 집의 가격(Target variable)
* bedrooms : 침실의 수
* bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )
* sqft_living : 주거 공간의 평방 피트(면적)
* sqft_lot : 부지의 평방 피트(면적)
* floors : 집의 층 수
* waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
* view : 집이 얼마나 좋아 보이는지의 정도
* condition : 집의 전반적인 상태
* grade : King County grading 시스템 기준으로 매긴 집의 등급
* sqft_above : 지하실을 제외한 평방 피트(면적)
* sqft_basement : 지하실의 평방 피트(면적)
* yr_built : 지어진 년도
* yr_renovated : 집을 재건축한 년도
* zipcode : 우편번호
* lat : 위도
* long : 경도
* sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
* sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

In [ ]:
# Loading packages
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc
import plotly.graph_objs as go
import plotly.offline as py
from plotly import tools

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test  = pd.read_csv('../input/test.csv')

In [ ]:
print("train.csv. Shape: ",df_train.shape)
print("test.csv. Shape: ",df_test.shape)

In [ ]:
df_train.head(10)

In [ ]:
df_test.head(10)

**ID

ID는 순서대로 고유번호이며, date는 모두 T000000이 달려 있다.

**Price
Price는 목적 변수이며, 분포는 다음과 같다.

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(df_train['price'])

목적 변수의 분포가 굉장히 치우쳐 있기 때문에, 정규화(log)를 취해줄 필요성이 있다.

정규화(log)를 취해주면 아래와 같이 이쁜 모양의 price가 쨘 하고 나온다.

In [ ]:
df_train['price'] = np.log1p(df_train['price'])
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(df_train['price'])

In [ ]:
df_train["bedrooms"].drop_duplicates()

**bedrooms

bedrooms는 단순하게 침실 개수로 추정된다.

In [ ]:
df_train["bathrooms"].drop_duplicates()

**bathrooms


bathrooms는 소수점이 존재하는데, 이는 미국에서는

full bath : 화장실, 세면대, 샤워실, 욕조
3/4 bath : 화장실, 세면대, 샤워실
half bath : 화장실, 세면대

과 같이 표기한다고 한다.  출처 : https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr/discussion/83533

예를 들어, 2.25가 값이라고 한다면, full bath + 3/4 bath + half bath 로 총 3개가 존재한다고 할 수 있다.

즉, 0~1은 1개, 1~2는 2개, 2~3는 3개로 추정 가능하다.

**sqlf_living / sqlf_lot

주거 평방 피트 / 부지 평방 피트

단위는 1 스퀘어피트 (sqlf)로 표현되어 있다.
1 sqlf = 0.0281평이다.

대략 1000sqlf은 대략 28평~32평 정도로 가늠하여 보면 된다.

sqlf_lot는 부지 평방 피트인데, 흔히 우리나라의 경우, 32평형 아파트는 일반적으로 실제로 쓰는 전용 면적은 25.7평 정도이다.

즉 sqlf_living은 실제 사용 면적 / sqlf는 총 부지의 면적으로 생각하면 된다.

In [ ]:
data = pd.concat([df_train['price'], df_train['sqft_living']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.regplot(x='sqft_living', y="price", data=data)

In [ ]:
df_train[df_train["sqft_living"]>13000]

sqft_living = 13540이 가장 떨어진 모습을 볼 수 있는데, 이상치로 판별해야하는 지에 대한 고민은 좀 더 필요해 보인다.

In [ ]:
df_train["floors"].drop_duplicates()

**floors

미국에서는 1.5 floors house plan을 보면 미국 또는 유럽 등에서 흔히 볼 수 있는 집이 검색 결과로 나온다.
예를 들어, 지붕이 삼각형으로 되어 있으며 다락방을 끼고 있는 집을 볼 수 있다.

즉 1.5 floors란 2층 집인데, 2층의 면적이 온전히 1층의 면적을 포함하지 못하는 수준의 집이라고 표현할 수 있겠다.



In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.boxplot(x="floors", y="price", data=df_train)
plt.title("Box Plot")
plt.show()

In [ ]:
df_train["waterfront"].drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.boxplot(x="waterfront", y="price", data=df_train)
plt.title("Box Plot")
plt.show()

**waterfront

waterfront == 1 이 리버뷰 인 것을 알 수 있다. 0은 물이 안보이는 집

그리고 대체적으로 리버뷰인 곳은 가격대가 꽤나 높게 형성되어 있다는 것을 알 수 있다.

In [ ]:
df_train["view"].drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.boxplot(x="view", y="price", data=df_train)
plt.title("Box Plot")
plt.show()

price가 높을 수록 수록 veiw 가 높다는 것을 확인 할 수 있다.



In [ ]:
df_train["condition"].drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.boxplot(x="condition", y="price", data=df_train)
plt.title("Box Plot")
plt.show()

price가 높을 수록 수록 condition 이 높다는 것을 확인 할 수 있다.

In [ ]:
df_train["grade"].drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.boxplot(x="grade", y="price", data=df_train)
plt.title("Box Plot")
plt.show()

price가 높을 수록 수록 grade 가 높다는 것을 확인 할 수 있다.

"1에서 13까지의 색인. 1-3은 건물 건축 및 디자인에 미치지 못하고 7은 평균 수준의 건축 및 디자인을, 11-13은 높은 수준의 건축 및 디자인을 지니고 있습니다." 라고 한다.

그런데, 3의 분포가 특이하다는 사실에 대해 조금 더 생각해볼 필요가 있어 보인다.

sqlf_above / sqlf_basement

sqlf_above는 지하실을 제외한 지상의 평방 피트 이며, sqlf_basement는 지하실의 평방 피트이다.

이때, sqlf_above는 부지 전체를 말하는 것이 아니라, sqlf_living, 즉 실제 전용 면적 중에서 지상인 것들
sqlf_basement는 실제 전용 면적 중에서 지하실인 면적을 뜻한다.

In [ ]:
df_train["yr_built"].describe()

In [ ]:
df_train[df_train["yr_renovated"]!=0]["yr_renovated"].describe()

**yr_bulit / yr_renovated

단순히 지어진 연도와 마지막으로 리모델링 된 연도다.

yr_built : 1900 ~ 2015
yr_renovated : 1940 ~ 2015    //   0은 한번도 리모델링 된 적 없다는 것을 뜻한다.

**zipcode : 우편번호
lat : 위도
long : 경도

https://www.kaggle.com/fulrose/map-visualization-with-folium-ing 
해당 커널을 참조하면,

강남, 서초, 반포 등과 같이 특정 지역에서의 집 값이 높다는 사실을 알 수 있다.

해당 위도와 경도를 google map에 검색해서 zip code를 검색하면 일치하는 결과를 얻을 수 있다.

해당 분포를 위도, 경도에 따라 나타내면, 분포와 집값을 볼 수 있다.

In [ ]:
df_train.plot(kind = "scatter", x = "long", y = "lat", alpha = 0.1, s = df_train["sqft_living"]*0.02, 
             label = "sqft_living", figsize = (10, 8), c = "price", cmap = plt.get_cmap("jet"), colorbar = True, sharex = False)

* sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
* sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)



https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr/discussion/83533
https://geodacenter.github.io/data-and-lab//KingCounty-HouseSales2015/
등에 따른 정의

* sqft_living15 : 이웃 15가구의 평균을 계산한 값
* sqft_lot15 : 이웃 15가구의 평균을 계산한 값

sqft_lot15의 값과 sqft_lot의 값이 같을 때, 아파트가 아니냐는 의견이 존재했다.
하지만, 해당 값의 위도와 경도를 확인한 결과 아파트는 아니었다..

In [ ]:
df_train[["sqft_living","sqft_lot","sqft_living15","sqft_lot15","yr_renovated","lat","long"]].head(30)

yr_renovated의 유무에 따라서 sqft_lot 값과 sqft_15의 값이 변한다는 상관관계는 확인하기 힘들다.

어떤 값을 의미하는 지 좀 더 탐색이 필요해 보인다.

총 정리

****ID : 집을 구분하는 번호**

   고유 번호로 이루어져 있음 
  
****date : 집을 구매한 날짜**

   T000000 제외

****price : 집의 가격(Target variable)**


****bedrooms : 침실의 수**

bedrooms는 단순하게 침실 개수로 추정된다.

****bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )**

full bath : 화장실, 세면대, 샤워실, 욕조 3/4 bath : 화장실, 세면대, 샤워실 half bath : 화장실, 세면대     

예를 들어, bathrooms의 값이 2.25라고 한다면, 집 안에 full bath하나, 3/4 bath하나, half bath 하나가 있을 것으로 유추할 수 있다.

출처 : https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr/discussion/83533


****sqft_living : 주거 공간의 평방 피트(면적)**

단위는 1 스퀘어피트 (sqlf)로 표현되어 있으며 1 sqlf = 0.0281평이다.

sqlf_living은 실제 사용 면적이다. sqlf는 총 부지의 면적으로 생각하면 된다.

****sqft_lot : 부지의 평방 피트(면적)**

sqlf_lot는 부지 평방 피트인데, 흔히 우리나라의 경우, 32평형 아파트는 일반적으로 실제로 쓰는 전용 면적은 25.7평 정도이다.

sqlf_lot는 총 부지의 면적으로 생각하면 된다.


****floors : 집의 층 수**

집의 층 수로는 1.0 / 1.5 / 2.0 / 2.5 / 3.0 / 3.5 의 변수 값이 존재한다.

이 때, 0.5가 달려있는 것들은 온전한 층이 아닌 다락방 수준의 방이 존재할 때를 말한다.

해당 지역인 시애틀, 즉 미국에서 꽤나 흔하게 볼 수 있는 가정집은 1.5층의 건물으로, 지붕이 삼각형으로 되어 있으면서 다락방이 딸려 있는 경우이다.

****waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)**

0이 없다는 뜻 이고 1이 있다는 뜻이다.


****view : 집이 얼마나 좋아 보이는지의 정도**

1~5로 이루어져 있으며, 집의 외관에 대한 평가로 생각할 수 있다.

view 값은 높을 수록 좋다.

****condition : 집의 전반적인 상태**

1~5로 이루어져 있으며, 집의 전반적인 상태에 대한 평가다.

condition 값은 높을 수록 좋다.


****grade : King County grading 시스템 기준으로 매긴 집의 등급**

1~13의 값으로 이루어져 있으며,. 1-3은 건물 건축 및 디자인에 미치지 못하고 7은 평균 수준의 건축 및 디자인을, 11-13은 높은 수준의 건축 및 디자인을 지니고 있다고 대략적으로 보면 된다.
높을 수록 높은 등급이다.

****sqft_above : 지하실을 제외한 평방 피트(면적)**

sqlf_above는 지하실을 제외한 지상의 평방 피트이다.
이때, sqlf_above는 부지 전체를 말하는 것이 아니라, sqlf_living, 즉 실제 전용 면적 중에서 지상인 것들을 의미한다.

****sqft_basement : 지하실의 평방 피트(면적)**

sqlf_basement는 지하실의 평방 피트이다.
sqlf_basement는 실제 전용 면적 중에서 지하실인 면적을 뜻한다.

****yr_built : 지어진 년도**

1900~2015

****yr_renovated : 집을 재건축한 년도**

1940~2015
0은 한번도 재건축이 된 적 없는 집을 의미한다.

****zipcode : 우편번호**

실제 값을 나타냄, 위도, 경도와 일치 ( google map에서 확인 가능 )

****lat : 위도**

****long : 경도**


****sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
**sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)**

또 다른 해석 존재

****sqft_living15 : 이웃 15가구의 평균을 계산한 값
**sqft_lot15 : 이웃 15가구의 평균을 계산한 값**

2. 상관관계 분석

범주형 범수가 포함되어 있으므로, 스피어만 상관계수를 활용해 본다.

In [ ]:
from scipy.stats import spearmanr

df_train_noid = df_train.drop("id",1)

plt.figure(figsize=(21,21))
sns.set(font_scale=1.25)
sns.heatmap(df_train_noid.corr(method='spearman'),fmt='.2f', annot=True, square=True , annot_kws={'size' : 15})

단순하게, 목적 변수인 price와 연관이 높은 순으로 배열해보면 다음과 같다.

In [ ]:
cor = df_train_noid.corr(method='spearman')
cor["price"].nlargest(n=20).index

3. 데이터 전처리

3-1 이상치 제거

sqft_living의 값입니다.

In [ ]:
df_train[df_train["sqft_living"]>13000]

해당 값의 위치가 집이 넓음에 비해 가격이 낮은 것은, 해당 위도의 위치에 따른 것이라고 생각되어, 이상치로 규정하지 않겠습니다. (아래의 map 확인)

grade에서 이상치로 보여졌던 값들에 대해서 살펴보겠습니다.

In [ ]:
data = pd.concat([df_train['price'], df_train['grade']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='grade', y="price", data=data)

In [ ]:
df_train[df_train["grade"]==3]

아래의 map을 보면, id=2302인 경우, map의 대략적인 위치가 낮게 형성되어 있는 곳이고, sqft 또한 작은데도 불구하고, price가 높게 책정되었다는 것으로 이상치로 규정하고 제거하겠습니다.

반면, id = 4123인 경우에는, 위도가 47.5보다 조금 위인 경우로, 꽤나 중심지로 보여지기 때문에, 제거하지 않겠습니다.

In [ ]:
df_train.loc[(df_train['price']>14.7) & (df_train['grade'] == 8)]

In [ ]:
df_train.loc[(df_train['price']>15.5) & (df_train['grade'] == 11)]

두 값 모두, date가 꽤나 최신이며, 집의 위치가 중심지로서 보여지기 때문에 제거하지 않습니다.

In [ ]:
df_train.plot(kind = "scatter", x = "long", y = "lat", alpha = 0.1, s = df_train["sqft_living"]*0.02, 
             label = "sqft_living", figsize = (10, 8), c = "price", cmap = plt.get_cmap("jet"), colorbar = True, sharex = False)

In [ ]:
df_train = df_train.loc[df_train['id']!=2302]

3-2 변수 정규화

출처. https://www.kaggle.com/kcs93023/2019-ml-month-2nd-baseline

위 커널을 활용하여, 분포가 치우쳐져 있는 변수들에 대해 정규화를 진행합니다.

In [ ]:
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    df_train[c] = np.log1p(df_train[c].values)
    df_test[c] = np.log1p(df_test[c].values)

3-3 변수 수정

먼저, date 뒤에 있는 T000000을 제외합니다.

In [ ]:
for df in [df_train,df_test]:
    df['date'] = df['date'].apply(lambda x: x[0:8])

4. 변수 생성

In [ ]:
df_train.head(4)

In [ ]:
a = [(1,3)]
a

In [ ]:
for df in [df_train,df_test]:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15']
    df['been_renovated'] = df['yr_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')


In [ ]:
for df in [df_train,df_test]:
    df['location'] = zip(df['lat'],df['long'])# 위치 변수

In [ ]:
df_train["location"][1]